# Clase 6 (OSS): Integración con **LangChain + Ollama**

Notebook *100% open‑source*. Usa tu `.env` existente:

```env
MODEL=qwen2.5:7b-instruct
OLLAMA_BASE_URL=http://localhost:11434
TEMPERATURE=0.0
```

Requisitos previos:

```bash
uv add "langchain>=0.3" "langchain-ollama>=0.3" "python-dotenv>=1.0"
uv add jupyter ipykernel --dev
uv run python -m ipykernel install --user --name my-course-agent
```

Luego abre con:

```bash
uv run jupyter lab
```


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=os.getenv("MODEL", "qwen2.5:7b-instruct"),
    base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
    temperature=float(os.getenv("TEMPERATURE", "0.0")),
)

resp = llm.invoke("Hola, hola, ¿cómo estás?")
resp.pretty_print()

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

msg_1 = SystemMessage(content="Eres un asistente útil que responde en español.")
msg_2 = HumanMessage(content="Me llamo Juan.")
msg_3 = AIMessage(content="Hola Juan, ¿cómo estás?")
msg_4 = HumanMessage(content="¿Cómo me llamo?")
history = [msg_1, msg_2, msg_3, msg_4]

resp = llm.invoke(history)
resp.pretty_print()

In [ ]:
def init_local_llm(model: str | None = None, temperature: float | None = None):
    from langchain_ollama import ChatOllama
    return ChatOllama(
        model=model or os.getenv("MODEL", "qwen2.5:7b-instruct"),
        base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
        temperature=float(temperature if temperature is not None else os.getenv("TEMPERATURE", "0.0")),
    )

llm_qwen = init_local_llm("qwen2.5:7b-instruct", temperature=0.0)
models = [("Qwen2.5:7b", llm_qwen)]

try:
    llm_llama = init_local_llm("llama3.1:8b-instruct", temperature=0.0)
    models.append(("Llama3.1:8b", llm_llama))
except Exception:
    pass

for name, m in models:
    try:
        print(f"== {name} ==")
        m.invoke("Dime un tip de Python en una línea.").pretty_print()
    except Exception as e:
        print(f"{name} no disponible: {e}")